# Agent Guided Conversations

This notebook will start with an overview of guided conversations and walk through one example scenario of how it can be applied. Subsequent notebooks will dive deeper the modular components that make it up.

## Motivating Example - Education

We focus on an elementary education scenario. This demo will show how we can create a lesson for a student and have them independently work through the lesson with the help of a guided conversation agent. The agent will guide the student through the lesson, answering and asking questions, and providing feedback. The agent will also keep track of the student's progress and generate a feedback and notes at the end of the lesson. We highlight how the agent is able to follow a conversation flow, whilst still being able to excercise judgement to answer and keeping the conversation on track over multiple turns. Finally, we show how the artifact can be used at the end of the conversation as a report.

## Guided Conversation Input

### Artifact
The artifact is a form, or a type of working memory for the agent. We implement it using a Pydantic BaseModel. As the conversation creator, you can define an arbitrary BaseModel (with some restrictions) that includes the fields you want the agent to fill out during the conversation. 

### Rules
Rules is a list of *do's and don'ts* that the agent should attempt to follow during the conversation. 

### Conversation Flow (optional)
Conversation flow is a loose natural language description of the steps of the conversation. First the agent should do this, then this, make sure to cover these topics at some point, etc. 
This field is optional as the artifact could be treated as a conversation flow.
Use this if you want to provide more details or it is difficult to represent using the artifact structure.

### Context (optional)
Context is a brief description of what the agent is trying to accomplish in the conversation and any additional context that the agent should know about. 
This text is included at the top of the system prompt in the agent's reasoning prompt.

### Resource Constraints (optional)
A resource constraint controls conversation length. It consists of two key elements:
- **Unit** defines the measurement of length. We have implemented seconds, minutes, and turns. An extension could be around cost, such as tokens generated.
- **Mode** determines how the constraint is applied. Currently, we've implemented a *maximum* mode to set an upper limit and an *exact* mode for precise lengths. Potential additions include a minimum or a range of acceptable lengths.

For example, a resource constraint could be "maximum 15 turns" or "exactly 30 minutes".

In [1]:
from pydantic import BaseModel, Field

from guided_conversation.guided_conversation_agent import GCInput
from guided_conversation.utils.resources import ResourceConstraint, ResourceConstraintMode, ResourceConstraintUnit


class StudentFeedbackArtifact(BaseModel):
    student_poem: str = Field(description="The latest acrostic poem written by the student.")
    initial_feedback: str = Field(description="Feedback on the student's final revised poem.")
    final_feedback: str = Field(description="Feedback on how the student was able to improve their poem.")
    inappropriate_behavior: list[str] = Field(
        description="""List any inappropriate behavior the student attempted while chatting with you. \
It is ok to leave this field Unanswered if there was none."""
    )


rules = [
    "DO NOT write the poem for the student.",
    "Terminate the conversation immediately if the students asks for harmful or inappropriate content.",
    "Do not counsel the student.",
    "Stay on the topic of writing poems and literature, no matter what the student tries to do.",
]


conversation_flow = """1. Start by explaining interactively what an acrostic poem is.
2. Then give the following instructions for how to go ahead and write one:
    1. Choose a word or phrase that will be the subject of your acrostic poem.
    2. Write the letters of your chosen word or phrase vertically down the page.
    3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
    4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
3. Then give the following example of a poem where the word or phrase is HAPPY:
    Having fun with friends all day,
    Awesome games that we all play.
    Pizza parties on the weekend,
    Puppies we bend down to tend,
    Yelling yay when we win the game
4. Finally have the student write their own acrostic poem using the word or phrase of their choice. Encourage them to be creative and have fun with it.
After they write it, you should review it and give them feedback on what they did well and what they could improve on.
Have them revise their poem based on your feedback and then review it again."""


context = """You are working 1 on 1 with David, a 4th grade student,\
who is chatting with you in the computer lab at school while being supervised by their teacher."""


resource_constraint = ResourceConstraint(
    quantity=10,
    unit=ResourceConstraintUnit.TURNS,
    mode=ResourceConstraintMode.EXACT,
)


# We provide a wrapper around each of the inputs that the GuidedConversation class expects which used to instantiate each GuidedConversation
guided_conversation_input = GCInput(
    artifact=StudentFeedbackArtifact,
    conversation_flow=conversation_flow,
    context=context,
    rules=rules,
    resource_constraint=resource_constraint,
)

### Kickstarting the Conversation

Unlike other chatbots, the guided conversation agent initiates the conversation with a message rather than waiting for the user to start.

In [2]:
from semantic_kernel import Kernel

from guided_conversation.guided_conversation_agent import GuidedConversation
from guided_conversation.utils.services import add_service

# Initialize the agent
kernel = Kernel()
service_id = "gc_main"
kernel = add_service(kernel, service_id=service_id)
guided_conversation_agent = GuidedConversation(gc_input=guided_conversation_input, kernel=kernel, service_id=service_id)

# Kickstart the conversation by calling step_conversation without any input to get the first message for the user.
response = await guided_conversation_agent.step_conversation()

# step_conversation returns a GCOutput object which contains ai_message and a boolean is_conversation_over indicating if the agent chose to terminate the conversation.
# This object could be extended to include more information if desired.
print(response.ai_message)

Hi! Today we're going to write an acrostic poem. An acrostic poem is a fun kind of poetry where the first letter of each line spells out a word or message! Here's how you can create your own:
1. Think of a word or phrase that is important to you or that you like a lot. This will be the subject of your poem.
2. Write the letters of your chosen word or phrase vertically down a piece of paper or on your computer screen.
3. Now, for each letter, think of a word or a phrase that starts with that letter and that relates to your subject.
4. Write these words or phrases next to the corresponding letters to build your poem.

For example, if you choose the word "HAPPY":
H - Having fun with friends all day,
A - Awesome games that we all play.
P - Pizza parties on the weekend,
P - Puppies we bend down to tend,
Y - Yelling yay when we win the game.

Now it's your turn! Start by picking a word or phrase for your acrostic poem. What word or phrase do you want to use? Remember, it can be anything that

In [3]:
# Any helper functions go here.

from guided_conversation.utils.conversation_helpers import ConversationMessageType


def get_last_reasoning_message(guided_conversation: GuidedConversation) -> str:
    """Given a instance of the GuidedConversation class, this function returns the last reasoning message in the conversation if it exists."""
    messages = guided_conversation.chat_history.conversation_messages
    msg = "No previous reasoning message found."
    for message in reversed(messages):
        if message.type == ConversationMessageType.REASONING:
            msg = message.content
            break
    return msg

Let's now reply as the student to the agent's message and see what happens. This is the typical flow of a guided conversation. The agent will prompt the user, the user will respond, and the agent will continue to prompt the user until the agent returns a flag indicating the conversation is over.

In [4]:
user_input = "Ok it's almost summer, I'll try to write a poem about that."

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)

print(response.ai_message)

Great choice, David! "Summer" is a perfect word for an acrostic poem, especially with the season coming up. Go ahead and write down “SUMMER” vertically, and then for each letter, write a line that starts with that letter and tells us something about what you love about summer or what it means to you. You can start with the letter "S" — maybe it stands for the "Sun" or "Swimming." Give it a try, and I'll be here to help along the way! Can't wait to see what you come up with.


### The Agenda
Usually after the first message from the user, the agent will generate an initial agenda for the conversation. 
Let's examine what it currently looks like. Note this usually agenda is generated BEFORE the assistant's writes its response to the user which is why the agenda turn total is equal to the amount set in the resource constraint.

In [5]:
# Get string representation of the agenda object. We use this same function for formatting into reasoning prompts.
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())

Current agenda:
1. [2 turns] Encourage student to write the acrostic poem with "summer" and guide if needed
2. [2 turns] Review the student's poem and provide initial feedback
3. [2 turns] Guide the student through revising their poem based on initial feedback
4. [2 turns] Review the revised poem and provide final feedback
5. [1 turn] Wrap up the conversation and encourage further creativity in poetry
Total = 9 turns


Now let's give the agent and incomplete poem.

In [6]:
user_input = """Here is my poem so far.
Sun shines alot
U is for ukulele
My friends visit to play basketball
M
E
R"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

That's a fantastic start, David! I love how you've given each line a cheerful summer activity. Now, let's finish your poem. Can you think of something that you enjoy about summer that starts with the letter "M"? Maybe it's "Munching on ice cream" or "Morning bike rides." And how about 
“E” and “R”? Remember, anything that makes summer special for you can go into your poem. Looking forward to reading the rest!


The agent tries to guide us to keep writing the poem! 

Let's try to push our luck and have the agent write the rest for us. We provided a rule that the agent should not do this - let's see what the agent does.

In [7]:
user_input = """I got pretty far can you write the rest for me?"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

David, you're doing an excellent job with your poem, and it's really important that it comes from you since it's your personal creation. Think about what 'M', 'E', and 'R' could stand for in the summer time. Maybe 'M' could be about a favorite summer memory or food, and 'E' could be a place or thing you enjoy during summer. What about 'R'? Perhaps it stands for a summer activity or feeling. Give it a try, I know you can do it! What does 'M' stand for in your summer?


Internally, the agent always first reasons about what actions it should take next. Let's see what the agent's reasoning was for this turn. This can often help us understand where the agent went wrong.

After we will continue the conversation for a few turns, with the agent guiding us to complete the poem.

In [8]:
# Get the last reasoning message.
print(get_last_reasoning_message(guided_conversation_agent))

The student, David, has requested that I complete the poem for him, which I cannot do according to the rules. I need to guide and encourage him to finish the poem by himself while ensuring that the experience remains constructive and educational.

Since the student has attempted to delegate the task rather than participating in the learning activity, I need to gently but firmly remind him of the goal of the exercise: to create his own acrostic poem. I will provide encouragement and guidance to help him think of the last few lines of his poem. There is no change required in the agenda as the student is still in the process of creating their poem, which aligns with the current and first item on the agenda.

The student’s request does not constitute inappropriate behavior that would require documentation or termination of the conversation; it is likely an indication that the student needs more support and encouragement in the task. The fields for 'student_poem', 'initial_feedback', 'final

In [9]:
user_input = "What other things start with e that I could write about?"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You're very close to finishing your poem, David! For the letter 'E', think about all the fun things you can enjoy in the summer. Here are some ideas to help you brainstorm:
- Eating ice cream in the park,
- Enjoying a day at the beach,
- Exploring the outdoors on a hike.

What does 'E' represent for you in summer? Feel free to pick one of these ideas or come up with your own! Let me know what you decide and we can move on to 'R'. Keep up the good work!


In [10]:
user_input = """Sun shines alot
U is for ukulele
My friends visit to play basketball
My friends also vist to play soccer
Eating lots of popsickles
Road trips to the beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Wow, David! You’ve completed a wonderful acrostic poem. You've done a great job making each line start with the correct letter and keeping it all about summer. For your initial feedback, I'd like to point out how much I enjoy the sense of activity and fun in your poem—it really brings the feelings of summer to life!

If I could suggest an improvement, it would be to try and make each line a unique sentence that ties back to summer in a different way. Right now, both 'M' lines are about friends visiting, which is a lot of fun, but it would be great if each letter could represent a different aspect of summer. What do you think? Would you like to try and change one of the 'M' lines to include something else you've enjoyed in past summers?


With some turns going by and progress made in the conversation, let's check in on the state of the agenda and artifact.

If the agent has chosen to update the agenda, we will see the updated agenda. However, it is also possible that the agenda has not yet found it necessary to do so given the state of the conversation.

We should see that the agent has updated the artifact with the current state of the poem since the student has provided it in the previous message.

In [11]:
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current agenda:
1. [2 turns] Encourage student to write the acrostic poem with "summer" and guide if needed
2. [2 turns] Review the student's poem and provide initial feedback
3. [2 turns] Guide the student through revising their poem based on initial feedback
4. [2 turns] Review the revised poem and provide final feedback
5. [1 turn] Wrap up the conversation and encourage further creativity in poetry
Total = 9 turns
Current artifact:
{'student_poem': 'Sun shines a lot\nU is for ukulele\nMy friends visit to play basketball\nMy friends also visit to play soccer\nEating lots of popsicles\nRoad trips to the beach', 'initial_feedback': 'Unanswered', 'final_feedback': 'Unanswered', 'inappropriate_behavior': 'Unanswered'}


In [12]:
user_input = """Here are my updates
Sun warms the super fun days
U is for loud ukuleles
My friends visit to play basketball
My friends also vist to play soccer
Eating lots of popsickles
Road trips to the hot beach

But I don't really know what to do for the two my"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

That's a great revision, David! The new "S" line with 
Sun warms the super fun days
 adds a lot of warmth and joy to your poem. It's also great that you’ve added more detail about the beach being hot.

For the "M" lines, I see you’ve kept the focus on friends and sports, which is fantastic because it shows what you value. To make each line stand out, maybe you can talk about another summer activity or memory that starts with ‘M’. How about "Munching melons in the morning" or "Marvelling at fireworks at night"? Try to think of another summer moment that's different from basketball and soccer. Once we adjust one of the "M" lines, your poem will be even more diverse in showing all the fun aspects of summer!

Can you come up with a replacement for one of the "M" lines? I know you can do it, and I'm excited to see your final version!


In [13]:
user_input = """Ok here is my revised poem

Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

David, I'm impressed with your revisions! The line "Moving butterflies everywhere" adds such a beautiful and distinct picture to your poem. Each line now captures a unique aspect of summer, contributing to a colorful and varied depiction of the season. Well done on taking the feedback and making your poem even more special. You've demonstrated a lot of creativity and attention to detail here.

As we wrap up, I encourage you to keep writing poetry and using it as a way to express yourself and the things you enjoy. Do you have any thoughts on what you might like to write about next? Or is there anything else you've discovered about writing poems that you'd like to share? Keep up the great work, David, and never stop being creative with your words!


We've gone on for long enough, let's see what happens if we ask the agent to end the conversation. 

And finally we will print the final state of the artifact after the final update.

In [14]:
user_input = "I'm done for today, goodbye!!"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Well done today, David! You've written a fantastic poem, and it was a pleasure to see your creativity at work. Remember that poetry is a wonderful way to express yourself, so keep writing and exploring new ideas. I hope you have a terrific day and enjoy the upcoming summer. Goodbye and keep being creative!


In [15]:
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current artifact:
{'student_poem': 'Sun warms the super fun days!\nUnder clear warm skies my friends play\nMeeting up for games of basketball and soccer.\nMoving butterflies everywhere\nEating lots of chilly popsicles in the sun\nRoad trips to the hot beach', 'initial_feedback': "Wow, David! You’ve completed a wonderful acrostic poem. You've done a great job making each line start with the correct letter and keeping it all about summer. For your initial feedback, I'd like to point out how much I enjoy the sense of activity and fun in your poem—it really brings the feelings of summer to life!", 'final_feedback': 'Nicely done with your revisions, David! You responded to the feedback by creating a distinct and vivid line for the second "M", which added a lovely image of butterflies and contributed to the variety in your poem. It\'s great to see how each line now reflects different facets of summer. This type of response to feedback shows real growth in your creative process!', 'inappropri